## Initialization

In [20]:
import osmnx
import networkx as nx
import polyline
from heapq import heappop, heappush
from itertools import count
from datetime import datetime
import requests

In [21]:
import osmnx
import networkx as nx
import polyline
from heapq import heappop, heappush
from itertools import count
from datetime import datetime
import requests

#### FUNCTIONS ####


def api_profile(weather, profile):
    """Adjusts the current profile according to current weather and time conditions."""
    new_profile = profile

    now = datetime.now().hour + 8
    if now >= 24:
        now = now - 24

    # Removes 'flood_hazard' if weather is clear
    if weather not in [202, 212, 221, 502, 503, 504]:
        new_profile.pop("not_flood_hazard")

    # Removes 'lighting' if time is day
    if now not in [19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5]:
        new_profile.pop("lighting")

    return new_profile


def adjust_weight(length, row, profile):
    """Returns the weight value for the given 'edge/row' parameters based on 'length' and 'profile'."""
    weight = length
    modifier = 1
    for safety_factor, user_preference in profile.items():
        if row[safety_factor] == '0':
            modifier += user_preference
    return weight * modifier

#### PATH FINDING ####


def getCoordinates(route, nodes, origin, destination):
    """Returns coordinates of route as an array of tuples."""
    final_coords = []

    final_coords.append({
        'latitude': origin['y'],
        'longitude': origin['x']
    })

    for id in route:
        coord = {
            'latitude': nodes.filter(items=[id], axis=0).y.item(),
            'longitude': nodes.filter(items=[id], axis=0).x.item()
        }
        final_coords.append(coord)

    final_coords.append({
        'latitude': destination['y'],
        'longitude': destination['x']
    })

    return final_coords


def getPolyline(route, nodes):
    """Returns encoded polyline using the polyline library, requires input as an array of tuples."""
    coordinates = getCoordinates(route, nodes)
    return polyline.encode(coordinates)


def getRouteLength(route, graph):
    """Function to return the total length of the route in meters."""
    route_length = osmnx.utils_graph.get_route_edge_attributes(
        graph, route, attribute='length')
    return round(sum(route_length))


def getBearingString(degrees, name):
    """Convert bearing value to readable instruction."""
    instruction = None
    if degrees < 45:
        instruction = 'Head North '
    elif degrees < 135:
        instruction = 'Head East '
    elif degrees < 225:
        instruction = 'Head South '
    elif degrees < 315:
        instruction = 'Head West '
    else:
        instruction = 'Head North '
    if isinstance(name, list):
        name = ' '.join(name)
    if name == '':
        return instruction
    return instruction + 'along ' + name


def getManeuever(heading, true_bearing):
    """Get maneuever type based on bearing relative_bearing."""
    relative_bearing = abs(true_bearing - heading)
    if relative_bearing <= 45 or relative_bearing >= 315:
        return 'straight'
    return 'turn'


def getTurnDirection(heading, true_bearing, name):
    """Translate turn direction in bearings to string."""
    relative_bearing = true_bearing - heading
    if relative_bearing < 0:
        relative_bearing += 360
    instruction = None
    if relative_bearing <= 45 or relative_bearing >= 315:
        instruction = 'Continue Straight '
    if relative_bearing >= 45 and relative_bearing < 180:
        instruction = 'Turn Right '
    if relative_bearing > 180 and relative_bearing <= 315:
        instruction = 'Turn Left '
    if isinstance(name, list):
        name = ' '.join(name)
    if name == '':
        return instruction
    return instruction + 'onto ' + name


def getRouteDirections(route, nodes, graph, safety_factors):
    """Generates the step-by-step instructions in a given 'route'.
    'nodes' are used to evaluate coordinates per step
    'graph' are used to generate the bearing values to calculate for turn directions
    'safety_factors' is the profile mask used to collect edge attributes of the 'route' in the 'graph'
    """
    # generate edge bearings for graph
    bearings_graph = osmnx.bearing.add_edge_bearings(graph, precision=1)
    # Generate a dictionary of relevant keys of the route for directions
    steps = osmnx.utils_graph.get_route_edge_attributes(bearings_graph, route)

    direction = []
    safety_coverage_direction = []
    instruction = None
    maneuever = None  # depart - first step, turn - for any step in between, arrive - last step
    distance = None
    name = None
    before_maneuever = None
    before_name = None
    footway = None
    bearing_before = 0
    bearing_after = 0
    count = 0
    # Start parsing
    for count, step in enumerate(steps):
        present_factors = []
        before_name = name
        before_maneuever = maneuever
        name = step.get("name", "")
        footway = step.get("footway")
        distance = round(step.get("length"))

        for factor in safety_factors:
            if step.get(factor) == '1':
                present_factors.append(factor)

        # If the step is the first step
        if steps[0] == step:
            bearing_before = 0
            bearing_after = step.get("bearing")
            maneuever = 'depart'
            instruction = getBearingString(bearing_after, name)
            direction.append({'maneuever': maneuever,
                              'footway': footway,
                              'instruction': instruction,
                              'name': name,
                              #   'bearing_before': bearing_before,
                              #   'bearing_after': bearing_after,
                              'distance': distance,
                              'factors_present': present_factors,
                              'coordinates': [
                                  nodes.filter(
                                      items=[route[count]], axis=0).y.item(),
                                  nodes.filter(
                                      items=[route[count]], axis=0).x.item()
                              ]})
            
            safety_coverage_direction.append({
                              'distance': distance,
                              'factors_present': present_factors})
            continue
        
        # If the step is any steps in between the first and last step
        if steps[0] != step and steps[-1] != step:
            bearing_before = bearing_after
            bearing_after = step.get("bearing")
            maneuever = getManeuever(bearing_before, bearing_after)

            if footway == 'crossing':
                instruction = 'Cross the street'
            elif footway == 'footbridge':
                instruction = 'Cross the street via the footbridge'
            else:
                instruction = getTurnDirection(
                    bearing_before, bearing_after, name)

            safety_coverage_direction.append({'distance': distance,
                                              'factors_present': present_factors})

            if before_name == name and before_maneuever == maneuever:
                direction[-1]["distance"] += distance
                for factor in present_factors:
                    if factor not in direction[-1]["factors_present"]:
                        direction[-1]["factors_present"].append(factor)
                continue

            direction.append({'maneuever': maneuever,
                              'footway': footway,
                              'instruction': instruction,
                              'name': name,
                              #   'bearing_before': bearing_before,
                              #   'bearing_after': bearing_after,
                              'distance': distance,
                              'factors_present': present_factors,
                              'coordinates': [
                                  nodes.filter(
                                      items=[route[count]], axis=0).y.item(),
                                  nodes.filter(
                                      items=[route[count]], axis=0).x.item()
                              ]})
            continue

        # If the step is the last step
        if steps[-1] == step:
            bearing_before = bearing_after
            bearing_after = step.get("bearing")
            maneuever = 'arrive'
            instruction = getTurnDirection(
                bearing_before, bearing_after, name) + " and arrive at destination"
            safety_coverage_direction.append({
                'distance': distance,
                'factors_present': present_factors,
            })
            direction.append({'maneuever': maneuever,
                              'footway': footway,
                              'instruction': instruction,
                              'name': name,
                              #   'bearing_before': bearing_before,
                              #   'bearing_after': bearing_after,
                              'distance': distance,
                              'factors_present': present_factors,
                              'coordinates': [
                                  nodes.filter(
                                      items=[route[count]], axis=0).y.item(),
                                  nodes.filter(
                                      items=[route[count]], axis=0).x.item()
                              ]})

    return direction, safety_coverage_direction


def getSafetyFactorCoverage(steps, length, safety_factors, profile):
    """Evaluates the safety factor coverage for each safety factor for the route.
    The calculation is based on the coverage of the safety factor (in m) divided by the total distance (in m) of the route.
    """
    factor_coverage = {
        'not_flood_hazard': 0,
        'pwd_friendly': 0,
        'cctv': 0,
        'landmark': 0,
        'lighting': 0,
        'not_major_road': 0
    }
    temp = 0

    for step in steps:
        for factor in safety_factors:
            if factor in step['factors_present']:
                factor_coverage[factor] += step['distance']
            else:
                pass

    for factor in safety_factors:
        factor_coverage[factor] = round((factor_coverage[factor]/length)*100)

    for item in profile.keys():
        if item in factor_coverage.keys():
            temp += factor_coverage[item] * profile[item]

    if sum(profile.values()) == 0:
        temp = sum(factor_coverage.values())/6
    else:
        temp = temp/sum(profile.values())

    factor_coverage['average'] = round(temp)

    return factor_coverage

## PathFinder .py step-by-step

In [22]:
safety_factors = ['not_flood_hazard', 'pwd_friendly',
                      'cctv', 'landmark', 'lighting', 'not_major_road']
osmnx.settings.useful_tags_way = safety_factors + ['name', 'footway']

In [23]:
profile = {"not_flood_hazard": 1,
            "pwd_friendly": 1,
            "cctv": 1,
            "landmark": 1,
            "lighting": 1,
            "not_major_road": 1}

In [24]:
coordinates = [14.6329782086551, 121.09862254304959, 14.631177362576384, 121.0821332033224]

In [25]:
# comes from application request
origin = {
    "y": coordinates[0],  # 14.635749969867808,
    "x": coordinates[1]  # 121.09445094913893
}
destination = {
    "y": coordinates[2],  # 14.63056033942939,
    "x": coordinates[3]  # 121.09807731641334
}

In [26]:
params = {
    'lat': coordinates[0],
    'long': coordinates[1],
    'API_key': '998183354bb6d9e4f0bf9a1ce02a8014'
}

api_result = requests.get(
    f'https://api.openweathermap.org/data/2.5/weather?lat={params["lat"]}&lon={params["long"]}&appid={params["API_key"]}')

api_response = api_result.json()

weather_condition = api_response['weather'][0]['id']

In [27]:
graph = osmnx.graph_from_xml('marikina_complete.osm', simplify=False)

In [28]:
nodes, edges = osmnx.graph_to_gdfs(graph)

In [29]:
adjusted_profile = api_profile(weather_condition, profile)

In [30]:
edges['weight'] = edges.apply(
        lambda row: adjust_weight(row['length'], row, adjusted_profile), axis = 1
        )

In [31]:
final_graph = osmnx.graph_from_gdfs(
        osmnx.graph_to_gdfs(graph, edges=False),
        edges
    )

In [32]:
origin_node_id = osmnx.nearest_nodes(final_graph, origin['x'], origin['y'], return_dist=True)
destination_node_id = osmnx.nearest_nodes(final_graph, destination['x'], destination['y'], return_dist=True)

In [33]:
route = nx.bidirectional_dijkstra(
    final_graph, 
    origin_node_id[0], 
    destination_node_id[0], 
    weight='weight'
    )

shortest_route = nx.bidirectional_dijkstra(
        final_graph,
        origin_node_id[0],
        destination_node_id[0],
        weight='length'
    )

In [34]:
route = route[1]
shortest_route = shortest_route[1]

route_dir, route_safety_dir = getRouteDirections(
    route, nodes, graph, list(adjusted_profile.keys()))

shortest_route_dir, shortest_route_safety_dir = getRouteDirections(
    shortest_route, nodes, graph, list(adjusted_profile.keys()))

In [36]:
route_dir

[{'maneuever': 'depart',
  'footway': 'sidewalk',
  'instruction': 'Head North along City Hall Road',
  'name': 'City Hall Road',
  'distance': 17,
  'factors_present': ['landmark', 'lighting', 'not_major_road'],
  'coordinates': [14.6329862, 121.0986221]},
 {'distance': 16,
  'factors_present': ['pwd_friendly',
   'landmark',
   'lighting',
   'not_major_road']},
 {'maneuever': 'straight',
  'footway': 'sidewalk',
  'instruction': 'Continue Straight onto City Hall Road',
  'name': 'City Hall Road',
  'distance': 16,
  'factors_present': ['pwd_friendly',
   'landmark',
   'lighting',
   'not_major_road'],
  'coordinates': [14.6331344, 121.0986045]},
 {'distance': 76,
  'factors_present': ['pwd_friendly',
   'landmark',
   'lighting',
   'not_major_road']},
 {'distance': 4,
  'factors_present': ['pwd_friendly',
   'landmark',
   'lighting',
   'not_major_road']},
 {'distance': 87,
  'factors_present': ['pwd_friendly',
   'landmark',
   'lighting',
   'not_major_road']},
 {'maneuever': '